In [ ]:
from tsapix.envbase import *
from tsapix.utils import env_db_root

from tsapi.ibkr.qqq import *

In [ ]:
import yfinance as yf

qqqpx = yf.download('QQQ')
qqqpxpd = qqqpx.xs('QQQ', axis=1, level=1)

In [ ]:
# import os
# import pandas as pd

# ohlcv2root = env_db_root+"/QQQ/ohlcv2/"
# ohlcv2fs = sorted(os.listdir(ohlcv2root))
# ohlcv2fs_curs = [f for f in ohlcv2fs if ".csv" in f]
# # ohlcv2fs_cur
# _qqqpxpds = [pd.read_csv(ohlcv2root + ohlcv2fs_cur, index_col=0) for ohlcv2fs_cur in ohlcv2fs_curs]

# qqqpxpd = pd.DataFrame({
#     'Date': [ohlcv2fs_cur.replace(".csv", "") for ohlcv2fs_cur in ohlcv2fs_curs],
#     'Open': [_qqqpxpd['1'].values[0] for _qqqpxpd in _qqqpxpds],
#     'High': [_qqqpxpd['2'].max() for _qqqpxpd in _qqqpxpds],
#     'Low': [_qqqpxpd['3'].min() for _qqqpxpd in _qqqpxpds],
#     'Close': [_qqqpxpd['4'].values[-1] for _qqqpxpd in _qqqpxpds],
# }).set_index('Date')
# qqqpxpd

In [ ]:
qqqpxpd.index[-1]

In [ ]:
app = DataApp()
app.connect("127.0.0.1", 4001, 0)
threading.Thread(target=app.run).start()
time.sleep(1)


In [ ]:
rtlist = ['C', 'P']
# ltdlist = [f"{xdt} 16:00:00 US/Eastern" for xdt in [
#     (datetime.datetime.today() - datetime.timedelta(x)).strftime("%Y%m%d") for x in range(1, 2)
# ]]
ltdlist = [f"{xdt} 16:00:00 US/Eastern" for xdt in [
    (datetime.datetime.today() - datetime.timedelta(x)).strftime("%Y%m%d") for x in range(1,2)
]]
barsetting = [("1 D", "1 min", "TRADES")]
# barsetting = [("1 D", "5 mins", "TRADES")]

from itertools import product

_reqlist = list(product(rtlist, ltdlist, barsetting))

paramslog = {}

from tqdm import tqdm

for _req in tqdm(_reqlist):
    _rt, _ltd, _bst = _req
    _ltddate = _ltd.split(" ")[0]
    stkrange = range(int(qqqpxpd.loc[_ltddate]['Low'])-5, int(qqqpxpd.loc[_ltddate]['High'])+5)
    for _stk in stkrange:
        _contract = init_qqq_options(_ltddate, _rt, _stk)
        
        _reqid = app.nextId()
        paramslog[_reqid] = [_rt, _ltd, _bst, _ltddate, _stk]
        print("Processing", _reqid, paramslog[_reqid])
        app.reqHistoricalData(
            _reqid, 
            _contract, 
            _ltd, 
            _bst[0], 
            _bst[1], 
            _bst[2], 
            1, 
            1, 
            False, 
            [])
        
#         try:
#             get_qqq_bardatapd(app.datapool[_reqid], _reqid)
#         except:
#             print("extract datapd failed", _reqid)
        
#         if _reqid%10==0:
#             time.sleep(10)

In [ ]:
_bst[1].replace(" ", "")

In [ ]:
def get_qqq_bardatapd(rawdatalist, _reqid, suffix=None, savepath=None, paramslog={}):
    import re
    import pandas as pd
    import os
    barsdata = re.findall(
    "([0-9]{2}[:][0-9]{2}[:][0-9]{2}) US/Eastern, Open: ([.0-9]+), High: ([.0-9]+), Low: ([.0-9]+), Close: ([.0-9]+), Volume: ([.0-9]+)", 
    " ".join([str(x) for x in rawdatalist]))
    _rt, _ltd, _bst, _ltddate, _stk = paramslog[_reqid]
    frq = _bst[1].replace(" ", "")
    fpath = f'{savepath}/{_ltddate}/'
    os.makedirs(fpath, exist_ok=True)
    if suffix is None:
        pd.DataFrame(barsdata).to_csv(f'{fpath}/0DTE{_rt}{_stk}_{frq}.csv')
    else:
        pd.DataFrame(barsdata).to_csv(f'{fpath}/0DTE{_rt}{_stk}_{frq}.csv')

In [ ]:
strdatapool = {}
for k, v in app.datapool.items():
    strdatapool[k] = [str(x) for x in v]
    
save_root = env_db_root + "/QQQ/options/"
os.makedirs(save_root, exist_ok=True)

for k, v in strdatapool.items():
    _suffix = "{}{}{}_{}".format(paramslog[k][3], paramslog[k][0], paramslog[k][4], paramslog[k][2][1])
    get_qqq_bardatapd(v, k, _suffix, 
                      savepath=save_root, paramslog=paramslog)

    

# Fake Options Data ...

In [ ]:
import pandas as pd
import os
from tsapi.univ_utils import save_as_yaml

qqqfroot = env_db_root + "/QQQ/"

ohlcvroot = env_db_root + "/QQQ/ohlcv2/"
optionsroot = env_db_root + "/QQQ/options/"

In [ ]:
ohlcvfs = [f.replace(".csv", "") for f in os.listdir(ohlcvroot) if ".csv" in f]

In [ ]:
optionsfs = [f for f in os.listdir(optionsroot) if "." not in f]

In [ ]:
no_options_fs = sorted([f for f in ohlcvfs if f not in optionsfs])

In [ ]:
save_as_yaml(f"{qqqfroot}/fake_options_fs.yaml", no_options_fs)
# no_options_fs

In [ ]:
# sample_optionsf1 = pd.read_csv(optionsroot+"20250718"+"/0DTEP560_1min.csv", index_col=0)
# sample_optionsf2 = pd.read_csv(optionsroot+"20250718"+"/0DTEC560_1min.csv", index_col=0)

reqId: -1, errorCode: 2107, errorString: HMDS data farm connection is inactive but should be available upon demand.ushmds, orderReject: 


In [ ]:
import os

for f in no_options_fs:
    froot = optionsroot+f
    os.makedirs(froot, exist_ok=True)
    for i in range(3):
        sample_optionsf1.to_csv(froot+f"/0DTEP{560+i}_1min.csv")
        sample_optionsf2.to_csv(froot+f"/0DTEC{560+i}_1min.csv")